<a href="https://colab.research.google.com/github/npsquared/3253_TermProject/blob/master/data_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
img_size = (28, 28, 1)
layer_sizes = (128, 64)
n_filters = (32, 64, 128)
kernel_sizes = (7, 3, 3)
pool_size = 2
drop_prob = 0.5


n_classes = 345   #output_dim/units is equal to the number of image classes

In [0]:
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten



In [0]:
model = Sequential()



In [0]:
# conv layer 1
model.add(Convolution2D(filters = 32,
                        kernel_size = (3,3),
                        activation = "relu",
                        input_shape = (28,28,1)))



In [0]:
# conv layer 2
model.add(Convolution2D(filters = 32,
                        kernel_size = (3,3),
                        activation = "relu"))



In [0]:
# pooling layer 1
model.add(MaxPooling2D(pool_size=(2,2)))



In [0]:
# deactivating some elements
model.add(Dropout(.20))



In [0]:
# conv layer 3
model.add(Convolution2D(filters = 64,
                        kernel_size = (3,3),
                        activation = "relu"))
                        



In [0]:
# conv layer 4
model.add(Convolution2D(filters = 64,
                        kernel_size = (3,3),
                        activation = "relu"))
                       



In [0]:
# pooling layer 2
model.add(MaxPooling2D(pool_size=(2,2)))



In [0]:
# deactivating some elements
model.add(Dropout(.1))



In [0]:
# flatten layer
model.add(Flatten())



In [0]:
# fully connected layer
model.add(Dense(512, activation="relu"))
model.add(Dense(units=345, activation="softmax"))

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 4, 4, 64)         

### Generating additional training and test data using ImageDataGenerator

In [0]:
train_path = "datasets/data/train_set/"
test_path = "datasets/data/test_set/"

data_file_path = "datasets/data/numpy_bitmap"

In [0]:
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=False,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


In [0]:
%%time

training_set = train_datagen.flow_from_directory(
        train_path,
        target_size=(28, 28),
        color_mode="grayscale",
        batch_size=128,
        class_mode='categorical')



Found 2760000 images belonging to 345 classes.


In [0]:
%%time

test_set = test_datagen.flow_from_directory(
        test_path,
        target_size=(28, 28),
        color_mode="grayscale",
        batch_size=128,
        class_mode='categorical') 



Found 690000 images belonging to 345 classes.
Wall time: 25.1 s


### Training the model

In [0]:
model.compile(optimizer = "nadam", loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:
save_cb = ModelCheckpoint("conv_doodle_model.h5",
                          save_best_only=True)

In [0]:
%%time

history = model.fit_generator(
            training_set,
            steps_per_epoch=12000,
            epochs=20,
            validation_data=test_set,
            validation_steps=3000, 
            callbacks=[save_cb])



Epoch 1/20
 1939/12000 [===>..........................] - ETA: 44:10 - loss: 4.6829 - accuracy: 0.0844

In [0]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1);

### Saving model

In [0]:
model.save("model_new.h5")


In [0]:
model_json = model.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)

In [0]:
# convolution layer 1
#model.add(Convolution2D(filters = 32,
#                        kernel_size = (3,3),
#                        activation = "relu",
#                        input_shape = (28,28,1)))


In [0]:
# convolution layer 2
#model.add(Convolution2D(filters = 32,
#                        kernel_size = (3,3),
#                        activation = "relu"))


In [0]:
# maxpooling layer 1
#model.add(MaxPooling2D(pool_size=(2,2)))


In [0]:
# deactivating some elements to avoid overfitting
#model.add(Dropout(.20))


In [0]:
# convolution layer 3
#model.add(Convolution2D(filters = 64,
#                        kernel_size = (3,3),
#                        activation = "relu"))
                        

In [0]:
# convolution layer 4
#model.add(Convolution2D(filters = 64,
#                        kernel_size = (3,3),
#                        activation = "relu"))


In [0]:
# maxpooling layer 2
#model.add(MaxPooling2D(pool_size=(2,2)))


In [0]:
# deactivating some elements
#model.add(Dropout(.1))


In [0]:
# flattening layer
#model.add(Flatten())

# fully connected layer
#model.add(Dense(256, activation="relu"))
#model.add(Dense(output_dim=5, activation="softmax"))  #output_dim is the number of classes used for classification
                                                      # i.e the image categories  


